## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---

# Sources

In [1]:
#extracting frames from video:  https://stackoverflow.com/questions/33311153/python-extracting-and-saving-video-frames
#a.	Real time lane detection for autonomous vehicles, Assidiq et. al.
#b.	Saad Bedros, Hough Transform and Thresholding lecture, University of Minnesota 
#c.	Lane detection techniques review, Kaur and Kumar
#d.	An Adaptive Method for Lane Marking Detection Based on HSI Color Model, Tran and Cho
#e.	LANE CHANGE DETECTION AND TRACKING FOR A SAFE-LANE APPROACH IN REAL TIME VISION BASED NAVIGATION SYSTEMS, Somasundaram, Ramachandran, Kavitha
#f.	A Robust Lane Detection and Departure Warning System, Mrinal Haloi and Dinesh Babu Jayagopi
#g.	Steerable filters
#h.	A layered approach to robust lane detection at night, Hayes and Pankati
#i.	SHADOW DETECTION USING COLOR AND EDGE INFORMATION
#j. fillpoly example:  https://www.programcreek.com/python/example/89415/cv2.fillPoly
#k. search around poly from coursework.  I did it in y, and this is convenient in x

In [2]:
#command line functions
#os.rmdir('../Undistorted Test Images')
#os.mkdir('../Undistorted_Test_Images')
#os.remove('../overpass.mp4')
#os.remove('../pavement.mp4')
#os.remove('../leaves.mp4')
#os.remove('../shadows.mp4')
#os.remove('../test_images/undistorted_straight_lines2.jpg')

Notes:
grayscale - doesn't do well on bright roads.  I tried using red instead.
magnitude - does great on black road, even way out to a distance, and seems to handle shadows
yellow with s and h - works well, but not out to a distance, even on changing road and can't handle shadows
white with l, s, and r - almost as good as magnitude on black roads, much better on imperfect roads
shadows - sobel_y doesn't do so well, but sobel_x and magnitude are pretty good 
hough transform would be good to dynamically find perspective transform
use the dR as a separate channel to find additiona lane markings
run windows on other found lines
it's very slow.  needs a lot of speeding up
adjacent lanes


# Import libraries

In [148]:
import numpy as np
import os
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib qt
from moviepy.editor import VideoFileClip
from IPython.display import HTML
from scipy.signal import argrelextrema


# Helper functions

In [161]:
def cal_undistort(img, objpoints, imgpoints):
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist


def threshold(image, thresh_min=0, thresh_max=255, scale = True):
    if scale:
        scaled = np.uint8(255*image/np.max(image)) # 4) Scale to 8-bit (0 - 255) then convert to type = np.uint8
    else:
        scaled = image
    binary_output = np.zeros_like(scaled)
    # Masking for region of interest
    mask = np.zeros_like(scaled)   
    ignore_mask_color = 100   
    imshape = scaled.shape
    vertices = np.array([[(0,690),(0, 420), (imshape[1], 420), (imshape[1],690)]], dtype=np.int32)
    cv2.fillPoly(mask, vertices, ignore_mask_color)

    
    binary_output[(scaled >= thresh_min) & (scaled <= thresh_max) & (mask > 0)] = 1
    return binary_output

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def dir_threshold(R_channel, sobel_kernel=3, thresh=(0, np.pi/2)):
    
    # Apply the following steps to img
    # 1) Take red channel
    # 2) Take the gradient in x and y separately
    # 3) Take the absolute value of the x and y gradients
    # 4) Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient 
    # 5) Create a binary mask where direction thresholds are met
    # 6) Return this mask as your binary_output image
    sobel_x = np.absolute(cv2.Sobel(R_channel, cv2.CV_64F, 1, 0,ksize=sobel_kernel))
    sobel_y = np.absolute(cv2.Sobel(R_channel, cv2.CV_64F, 0, 1,ksize=sobel_kernel))
    dir_grad = np.absolute(np.arctan2(sobel_y,sobel_x))
    #scaled_sobel = np.uint8(255*dir_grad/np.max(dir_grad)) # 4) Scale to 8-bit (0 - 255) then convert to type = np.uint8
    binary_output = np.zeros_like(dir_grad)
    
    # Masking for region of interest
    mask = np.zeros_like(dir_grad)   
    ignore_mask_color = 100   
    imshape = dir_grad.shape
    vertices = np.array([[(0,660),(0, 420), (imshape[1], 420), (imshape[1],660)]], dtype=np.int32)
    cv2.fillPoly(mask, vertices, ignore_mask_color)
        
    binary_output[(dir_grad >= thresh[0]) & (dir_grad <= thresh[1]) & (mask > 0)] = 1
    return binary_output

def find_lane_pixels_windows(binary_warped, leftx_base, rightx_base):
    # Create an output image to draw on and visualize the result
    windows = np.dstack((binary_warped, binary_warped, binary_warped))
    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []
    leftx = []
    lefty = []
    rightx = []
    righty = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        ### TO-DO: Find the four below boundaries of the window ###
        win_xleft_low = leftx_current - margin//2  # Update this
        win_xleft_high = leftx_current + margin//2  # Update this
        win_xright_low = rightx_current - margin//2  # Update this
        win_xright_high = rightx_current + margin//2  # Update this

        
        # Draw the windows on the visualization image
        cv2.rectangle(windows,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(windows,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        
        ### TO-DO: Identify the nonzero pixels in x and y within the window ###
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high))
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high))
        
        
        # Extract left and right line pixel positions
        good_leftx = nonzerox[good_left_inds]
        good_lefty = nonzeroy[good_left_inds]
        good_rightx = nonzerox[good_right_inds]
        good_righty = nonzeroy[good_right_inds]
        
        leftx.extend(good_leftx)
        lefty.extend(good_lefty) 
        rightx.extend(good_rightx)
        righty.extend(good_righty)
        
        
        if len(good_leftx) > minpix:
            leftx_current = int(np.mean(good_leftx))
        if len(good_rightx) > minpix:
            rightx_current = int(np.mean(good_rightx))
            
            ## Visualization ##
    # Colors in the left and right lane regions
    windows[lefty, leftx] = [255, 0, 0]
    windows[righty, rightx] = [0, 0, 255]
    
    return leftx, lefty, rightx, righty, windows

def search_around_poly(binary_warped, left_fit, right_fit):
    # HYPERPARAMETER
    # Choose the width of the margin around the previous polynomial to search
    # The quiz grader expects 100 here, but feel free to tune on your own!
    margin = 75

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    #for x, y in zip(nonzerox, nonzeroy):
        #print(x, y, left_fit, np.polyval(left_fit, y))
        
    ### TO-DO: Set the area of search based on activated x-values ###
    ### within the +/- margin of our polynomial function ###
    ### Hint: consider the window areas for the similarly named variables ###
    ### in the previous quiz, but change the windows to our new search area ###
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    
    
    ## Visualization ##
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]
    
    
    return leftx, lefty, rightx, righty, out_img


def calculate_poly(leftx, lefty, rightx, righty, binary_warped):
     ### TO-DO: Fit a second order polynomial to each using `np.polyfit` ###
    left_fit = [0,0,0]
    right_fit = [0,0,0]
    if ((lefty != [])&(leftx!=[])):
            left_fit = np.polyfit(lefty, leftx, 2)
    if ((righty != [])&(rightx!=[])):
        right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    #print(binary_warped.shape[0])
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left_fit` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty
      
        
    return left_fitx, right_fitx, ploty, left_fit, right_fit


def measure_curvature_real(leftx, lefty, rightx, righty, xm_per_pix, ym_per_pix):
    
 
    left_fit_cr = [0,0,0]
    right_fit_cr = [0,0,0]
    
    if ((leftx!=[]) & (lefty != [])):
        left_fit_cr = np.polyfit(np.array(lefty,dtype=int)*ym_per_pix, np.array(leftx, dtype=int)*xm_per_pix, 2)
    if ((rightx!=[]) & (righty!=[])):
        right_fit_cr = np.polyfit(np.array(righty, dtype=int)*ym_per_pix, np.array(rightx, dtype=int)*xm_per_pix, 2)
    
    #print('Right fit is %s,  Left fit is %s' %(right_fit_cr, left_fit_cr))
    
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = 720
        
    # Calculation of R_curve (radius of curvature)
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])

     
    return left_curverad, right_curverad

# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # should line be printed
        self.printme = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        # poly fit values of the last n fits of the line
        self.recent_coeffs = []
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.fit_prev = [np.array([False])]  
        #xvalues of the most recent fit
        self.fitx_points = [np.array([False])]
        #xvalues from the average polynomial self.best_fit
        self.best_fitx_points = [np.array([False])]
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None 
        #number of missed detections
        self.bad_sweeps = 31
        #threshold for missed detections before redoing windows
        self.threshold = 30
        
        
    
def Line_is_ok(found_x, found_y, ploty, fit_points, coeffs, lane_line):
    good_init = (fit_points[-1] > 200)  & (fit_points[-1] < 950)
    good_final = True
    if lane_line.recent_xfitted != []:
        good_final = (fit_points[0] < lane_line.best_fitx_points[0] + 50) & (fit_points[0] > lane_line.best_fitx_points[0] - 50)
    good_coeffs = True
    #initial X > 0 + margin
    #intial x between 300 and 950 fit_points[-1] is the firts
    #final x within 50 of previous final  fit_points[0]
    #if lane_line.recent_xfitted != []:
    #    good_init = (np.absolute(((fit_points[-1] - np.mean(lane_line.recent_xfitted))/np.mean(lane_line.recent_xfitted))) < .1)  
    #if lane_line.recent_coeffs != []:
     #   good_coeffs = ((np.absolute((coeffs[0] - lane_line.recent_coeffs[-1][0])/lane_line.recent_coeffs[-1][0]) < .1) &
      #                  (np.absolute((coeffs[1] - lane_line.recent_coeffs[-1][1])/lane_line.recent_coeffs[-1][1]) < .1) &
       #                 (np.absolute((coeffs[2] - lane_line.recent_coeffs[-1][2])/lane_line.recent_coeffs[-1][2]) < .1)) 
    #print('X fit: %s, %s, %s, %s' %(good_init,fit_points[-1], 
    #                            lane_line.recent_xfitted, 
    #                            (np.absolute(((fit_points[-1] - np.mean(lane_line.recent_xfitted))/np.mean(lane_line.recent_xfitted))))))
    #print('Coeffs fit: %s, %s' %(coeffs, lane_line.recent_coeffs))
    if (good_init & good_final):
        lane_line.detected = True
        lane_line.printme = True
        lane_line.bad_sweeps = 0
        lane_line.fitx_points = fit_points
        lane_line.best_fitx_points = fit_points
        lane_line.recent_xfitted.append(fit_points[0])
        lane_line.recent_coeffs.append(coeffs)
        lane_line.allx = found_x
        lane_line.ally = found_y
        lane_line.printme = True
        if (len(lane_line.recent_xfitted) > 8):
            del lane_line.recent_xfitted[:1]
            del lane_line.recent_xfitted[:1]
            del lane_line.recent_coeffs[:1]
            del lane_line.recent_coeffs[:1]
        lane_line.bestx = np.mean(lane_line.recent_xfitted)
        if (len(lane_line.recent_coeffs) >= 3):
            fit_mat = np.vstack((lane_line.recent_coeffs[-1],lane_line.recent_coeffs[-2],lane_line.recent_coeffs[-3]))
            lane_line.best_fit = [np.mean([fit_mat[0][0], fit_mat[1][0], fit_mat[2][0]]), 
                                  np.mean([fit_mat[0][1], fit_mat[1][1], fit_mat[2][1]]),
                                  np.mean([fit_mat[0][2], fit_mat[1][2], fit_mat[2][2]])]
            lane_line.best_fitx_points = lane_line.best_fit[0]*ploty**2 + lane_line.best_fit[1]*ploty + lane_line.best_fit[2]
            #print('best fit: %s' %lane_line.best_fit)
    if ((good_init == False) | (good_final == False)):
        if (lane_line.bad_sweeps < lane_line.threshold):
            lane_line.detected = False
            lane_line.printme = True
            lane_line.bad_sweeps += 1
        else:
            lane_line.__init__()
            lane_line.printme = False
    return 0

def lane_is_ok(left, right, ploty):
    distance_ok = False
    not_crossed = True
    if (left.printme & right.printme): 
        top_distance = np.polyval(right.recent_coeffs[-1], ploty[0]) - np.polyval(left.recent_coeffs[-1], ploty[0])
        bottom_distance = np.polyval(right.recent_coeffs[-1], ploty[-1]) - np.polyval(left.recent_coeffs[-1], ploty[-1])
        distance_ok = (top_distance/bottom_distance < 1.1) & (top_distance/bottom_distance > .9)

        not_crossed = (left.fitx_points[0] < right.fitx_points[0])
        
        top_slope_right = (2*right.recent_coeffs[-1][0]*ploty[-1] + right.recent_coeffs[-1][1])
        top_slope_left = (2*left.recent_coeffs[-1][0]*ploty[-1] + left.recent_coeffs[-1][1])
        bottom_slope_right = (2*right.recent_coeffs[-1][0]*ploty[0] + right.recent_coeffs[-1][1])
        bottom_slope_left = (2*left.recent_coeffs[-1][0]*ploty[0] + left.recent_coeffs[-1][1])

        top_slope_ok =  (top_slope_right/top_slope_left < 1.1) & (top_slope_right/top_slope_left > .9)
        bottom_slope_ok =  (bottom_slope_right/bottom_slope_left < 1.1) & (bottom_slope_right/bottom_slope_left > .9)
    #print('TOP - Right slope: %s, Left slope: %s' %(top_slope_right, top_slope_left))
    #print('BOTTOM - Right slope: %s, Left slope: %s' %(bottom_slope_right, bottom_slope_left))
    #print('top slope ok = %s' %top_slope_ok)
    #print('bottom slope ok = %s' %bottom_slope_ok)
    #print('distance ok = %s' %distance_ok)
    lane_ok = left.printme & right.printme & not_crossed#top_slope_ok & bottom_slope_ok 
    
    return lane_ok
   

# First, I'll compute the camera calibration using chessboard images

In [178]:
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        plt.imshow(img)
        #cv2.imshow('img',img)
        #cv2.waitKey(500)


# TODO: Write a function that takes an image, object points, and image points
# performs the camera calibration, image distortion correction and 
# returns the undistorted image
img = cv2.imread('camera_cal/calibration1.jpg')
undistorted = cal_undistort(img, objpoints, imgpoints)
cv2.imwrite('camera_cal/undistorted.jpg', undistorted)
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(undistorted)
ax2.set_title('Undistorted Image', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

print('done')


done


# Pipeline

In [179]:
def process_image(image):
    #initialize a bunch of variables that we'll use later.  you'll see them
    offset = 0
    i = 1
    sobel_kernel = 5
    lane_width = 3.7
    bad_sweeps_thresh = -1
    max_locations = [0]
    histogram =[]
    offset = -1
    curvature = -1
    

    #undistort the image using our camera undistortion matrix
    undistorted = cal_undistort(image, objpoints, imgpoints)
    
    #define color filters
    R = undistorted[:,:,0]
    G = undistorted[:,:,1]
    B = undistorted[:,:,2]
    hls = cv2.cvtColor(undistorted, cv2.COLOR_RGB2HLS)
    H = hls[:,:,0]
    L = hls[:,:,1]
    S = hls[:,:,2]

    blur = gaussian_blur(R, 5)  #add gaussian blur
    sobel_x = np.absolute(cv2.Sobel(blur, cv2.CV_64F, 1, 0,ksize=sobel_kernel))
    sobel_y = np.absolute(cv2.Sobel(blur, cv2.CV_64F, 0, 1,ksize=sobel_kernel))
    mag_grad = np.sqrt(np.power(sobel_x,2)+np.power(sobel_y,2))

    x_binary = threshold(sobel_x, 20, 60)
    y_binary = threshold(sobel_y, 20, 60)
    mag_binary = threshold(mag_grad, 35, 150)                      
    Canny_binary = cv2.Canny(R, 50, 150)  # add canny
    dir_binary = dir_threshold(R, sobel_kernel=3, thresh=(1,1.6))
    r_binary = threshold(R, 200,255, False)
    g_binary = threshold(G, 200,255, False)
    b_binary = threshold(B, 200,255, False)
    h_binary = threshold(H, 20, 100, False)
    s_binary = threshold(S, 90, 255, False)
    l_binary = threshold(L, 200, 255, False)

    
    ##This is where I combine the thresholds:  
    ##For edge detection, I use the red channel instead of the grayscale.  It does a much better job of picking up the lane lines.
    ##I look for vertical gradients with high magnitude that are picked up by the SobelX operator
    dir_interesting = ((mag_grad > 0) & (dir_binary > 0) & (x_binary > 0))    

    #Identify white lines: both S and the gradient of R do a good job of picking out white, when L is high
    white = (l_binary & (s_binary | dir_interesting))  

    #Identify yellow lines: combination of S and H gets yellow.  This was hinted in the lesson and developed in Tran's paper
    yellow = (s_binary & h_binary) 

    #Combined output of the thresholds
    combined = (white | yellow)

    #warp the image to bird's eye view    
    src = np.float32([[(200, 720), (545, 485), (742, 485), (1080, 720)]]) 
    dst = np.float32([[(400, 720), (400, 250), (850, 250), (850, 720)]])
    M = cv2.getPerspectiveTransform(src, dst)                     # d) use cv2.getPerspectiveTransform() to get M, the transform matrix
    img_size = (combined.shape[1], combined.shape[0])                 # e) use cv2.warpPerspective() to warp your image to a top-down view
    undistorted_warped = cv2.warpPerspective(undistorted, M, img_size, flags=cv2.INTER_LINEAR)
    warped = cv2.warpPerspective(combined, M, img_size, flags=cv2.INTER_LINEAR)
    
    #find lane points.  I have implemented both a window search and a polynomial sweep based on previous outcomes
    #this first if statement checks to see that the number of missed sweeps is below a threshold.  If so, we use our 
    #polynomial sweep.  If not, I force the window method again.
    if ((left_lane.bad_sweeps < left_lane.threshold) & (right_lane.bad_sweeps < right_lane.threshold)): 
        leftx, lefty, rightx, righty, found_points = search_around_poly(warped, left_lane.recent_coeffs[-1], right_lane.recent_coeffs[-1])
        left_lane_fitx_points, right_lane_fitx_points, ploty, left_lane_fit, right_lane_fit = calculate_poly(leftx, lefty, rightx, righty, warped)
        
        #two sanity checking functions.  I verify that each line is reasonable, and I verify that the lane itself is reasonble
        Line_is_ok(leftx, lefty, ploty, left_lane_fitx_points, left_lane_fit, left_lane)
        Line_is_ok(rightx, righty, ploty, right_lane_fitx_points, right_lane_fit, right_lane)
        lane_ok = lane_is_ok(left_lane, right_lane, ploty)
        
        #print('lane_ok = %s, right_lane_ok = %s, left_lane_ok = %s' %(lane_ok, right_lane.detected, left_lane.detected))
        
        # Generate a polygon to illustrate the search window area
        # And recast the x and y points into usable format for cv2.fillPoly()
        margin = 75
        window_img = np.zeros_like(found_points)
        left_line_window1 = np.array([np.transpose(np.vstack([left_lane_fitx_points-margin, ploty]))])
        left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_lane_fitx_points+margin, ploty])))])
        left_line_pts = np.hstack((left_line_window1, left_line_window2))
        right_line_window1 = np.array([np.transpose(np.vstack([right_lane_fitx_points-margin, ploty]))])
        right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_lane_fitx_points+margin, ploty])))])
        right_line_pts = np.hstack((right_line_window1, right_line_window2))

        # Draw the lane onto the warped blank image
        cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
        cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
        found_points = cv2.addWeighted(found_points, 1, window_img, 0.3, 0)
        
        fit = 'poly'
    
    #window search
    elif ((left_lane.bad_sweeps >= left_lane.threshold) | (right_lane.bad_sweeps >= right_lane.threshold)):
        fit = 'windows'
        
        ##this method falls somehwere between the convolution and histogram search shown in the lectures
        ##In the lesson, we separate the image into two and take the max on each side.  I wanted to find all possible
        ##lane markers, including those from adjacent lanes. So, instead of taking the max from each half of the image
        ##I ran sliding windows across the image to find peaks in the histogram.  I could have found sums in the windows
        ##which is the equivalent of convolution.  It works out to the same.  
        ##then, after I've found all potential lines, I take the nearest one on the left and the nearest one on the right
        ##for further analysis
        histogram = np.sum(warped[warped.shape[0]//2:,:], axis=0)
        #Find x locations of likely lines.  We will use these as starting points for the window search 
        maxs = [0]
        max_locations = [0]
        prev_max = 0
        count = 0
        bin = 10
        for j in range(1, histogram.shape[0], bin):
            maxim = np.amax(histogram[(j-1):(j+9)])
            if ((count <= 2) & (maxim > maxs[-1])  & (maxim > 20)):  #replace previous if adjacent and bigger
                maxs[-1] = maxim
                max_locations[-1]= (j+4)
                count = 0
            elif ((count > 2) & (maxim > prev_max) & (maxim > 20)):
                maxs.append(maxim)
                max_locations.append(j+4)
                count = 0
            elif (maxim <= prev_max):
                count +=1
            #print(maxim, prev_max, count, maxs, histogram[(j-1):(j+9)])
            prev_max = maxim

        max_locations = max_locations[1:]
        midpoint = np.int(histogram.shape[0]//2)

        left_lines = [x for x in max_locations if x < midpoint]
        right_lines = [x for x in max_locations if x > midpoint]
        nearest_left = 0
        nearest_right = 0
    
        ##sometimes we don't find a line.  This logic makes sure there are no errors from calls to an empty array
        if left_lines != []:
            nearest_left = left_lines[-1]
        if right_lines != []: 
            nearest_right = right_lines[0]
    
        ##the next function runs a window search on the warped image using the points picked up by the histogram to start
        leftx, lefty, rightx, righty, found_points = find_lane_pixels_windows(warped, nearest_left, nearest_right)
        
        ##calculate the polynomial based on the values returned from the window search
        left_lane_fitx_points, right_lane_fitx_points, ploty, left_lane_fit, right_lane_fit = calculate_poly(leftx, lefty, rightx, righty, warped)

        #two sanity checking functions.  I verify that each line is reasonable, and I verify that the lane itself is reasonble
        Line_is_ok(leftx, lefty, ploty, left_lane_fitx_points, left_lane_fit, left_lane)
        Line_is_ok(rightx, righty, ploty, right_lane_fitx_points, right_lane_fit, right_lane)
        lane_ok = lane_is_ok(left_lane, right_lane, ploty)
     
    
    #print('Left lane coeffs:  %s' %left_lane.recent_coeffs)
    #print('Right lane coeffs:  %s' %right_lane.recent_coeffs)
    #print('Left lane xs:  %s' %left_lane.recent_xfitted)
    #print('Right lane xs:  %s' %right_lane.recent_xfitted)    
    # Create an image to draw the lane lines on
    
    ##draw the lane lines and fill and polygon on a blank warp that can then be addweighted to the original image
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = []
    pts_right = []
    if left_lane.printme == True:  ##only draw the lane line if we've found it
        pts_left = np.vstack((left_lane.best_fitx_points,ploty)).astype(np.int32).T
        cv2.polylines(color_warp,  [pts_left],  False,  (0, 255, 0),  20)
    if right_lane.printme == True: ##only draw the lane line if we've found it
        pts_right = np.vstack((right_lane.best_fitx_points,ploty)).astype(np.int32).T
        cv2.polylines(color_warp,  [pts_right],  False,  (0, 255, 0),  20)
    if lane_ok == True:  ##only fill in the lane if both lines are valid
        pts = np.vstack((pts_left, pts_right[::-1]))
        cv2.fillPoly(color_warp, [pts], (0, 0, 255))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    Minv = cv2.getPerspectiveTransform(dst, src)                     # d) use cv2.getPerspectiveTransform() to get M, the transform matrix
    img_size = (combined.shape[1], combined.shape[0])                 # e) use cv2.warpPerspective() to warp your image to a top-down view
    newwarp = cv2.warpPerspective(color_warp, Minv, img_size, flags = cv2.INTER_LINEAR) 
    
    ##calculate curvature and offset, but only if the lane_ok variable is TRUE
    if lane_ok == True:
        ##the following functions caculate curvature and offset in meters
        # Define conversions in x and y from pixels space to meters
        ym_per_pix = 3/(405-288) # meters per pixel in y dimension
        xm_per_pix = lane_width/(right_lane.recent_xfitted[-1] - left_lane.recent_xfitted[-1]) # meters per pixel in x dimension

        # Calculate the radius of curvature in meters for both lane lines

        left_curve_real, right_curve_real = measure_curvature_real(left_lane.best_fitx_points, ploty, right_lane.best_fitx_points, ploty, xm_per_pix, ym_per_pix)
        curvature = (left_curve_real + right_curve_real)/2
        
        #calculate offset from lane center in pixels and in meters
        lane_pixels_hist = np.sum(newwarp[:,:,1][(newwarp.shape[0]-10):,:], axis=0)
        unwarped_hist_midpoint = np.int(lane_pixels_hist.shape[0]//2)
        right_lane_unwarped_pix_location = np.argmax(lane_pixels_hist[unwarped_hist_midpoint:]) + unwarped_hist_midpoint
        left_lane_unwarped_pix_location = np.argmax(lane_pixels_hist[:unwarped_hist_midpoint])
        unwarped_lane_midpoint = (right_lane_unwarped_pix_location + left_lane_unwarped_pix_location)//2
        pix_offset = newwarp.shape[1]//2 - unwarped_lane_midpoint
        offset = round(pix_offset * lane_width / (right_lane_unwarped_pix_location -  left_lane_unwarped_pix_location),2)
    
       
    #prepare the final image by combining the lane markers with the original image and the text
    result = cv2.addWeighted(undistorted, 1, newwarp, 0.3, 0)
    cv2.putText(result,'Curvature = %s' %(round(curvature,0)),(10,50), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),2,cv2.LINE_AA)
    cv2.putText(result,'Offset is %s meters from center of lane' %(offset),(10,150), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),2,cv2.LINE_AA)
    #cv2.putText(result,'Fit type is: %s, Lane ok = %s' %(fit, lane_ok),(10,200), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),2,cv2.LINE_AA)
    return result#, undistorted, combined, undistorted_warped, max_locations, newwarp, histogram, warped, found_points, color_warp







# Run on jpg

In [182]:
#file_list = os.listdir("test_images/challenge/")
#file_list = ['test1.jpg', 'test2.jpg', 'test3.jpg', 'test4.jpg', 'test5.jpg', 'test6.jpg', 'pavement_frame0.jpg','overpass_frame0.jpg', 'shadows_frame0.jpg']
#file_list = ['color_change_video_frame00.jpg', 'color_change_video_frame01.jpg', 'color_change_video_frame02.jpg']
#file_list = ['overpass_frame0.jpg', 'overpass_frame1.jpg', 'overpass_frame2.jpg']
file_list =['straight_lines1.jpg']
i = 3;
left_lane = Line()
right_lane = Line()
for name in file_list:
    print('reading %s' %(name))
    #read in the image
    image = mpimg.imread('./test_images/%s'  %(name))   #read in the image
    result, undistorted, combined, undistorted_warped, max_locations, newwarp, histogram, warped, found_points, color_warp = process_image(image)
    undistorted_BGR = cv2.cvtColor(undistorted,cv2.COLOR_BGR2RGB)
    cv2.imwrite('output_images/undistorted.jpg', undistorted_BGR)
    
    combined_out = np.dstack((combined, combined, combined))
    cv2.imwrite('output_images/combined_binary.jpg', combined_out)
    
    undistorted_warped_BGR = cv2.cvtColor(undistorted_warped,cv2.COLOR_BGR2RGB)
    cv2.imwrite('output_images/birds_eye.jpg', undistorted_warped_BGR)
    
    warped_out = np.dstack((warped, warped, warped))
    cv2.imwrite('output_images/warped_binary.jpg', warped_out)
    
    found_points_BGR = cv2.cvtColor(found_points,cv2.COLOR_BGR2RGB)
    color_fit = cv2.addWeighted(found_points, 1, color_warp, 0.3, 0)
    cv2.imwrite('output_images/color_fit.jpg', color_fit);
    
    result_BGR = cv2.cvtColor(result,cv2.COLOR_BGR2RGB)
    cv2.imwrite('output_images/output.jpg', result_BGR);
    
    plt.figure(1, figsize=(24, 12))
    plt.figure(1).tight_layout()
    plt.figure(1).suptitle('Combined Binary', fontsize=16)
    plt.imshow(combined)

    plt.figure(2, figsize=(24, 12))
    plt.figure(2).tight_layout()
    plt.figure(2).suptitle('Warped Binary', fontsize=16)
    plt.imshow(color_fit)

    
    plt.figure(i, figsize=(24, 12))
    plt.figure(i).tight_layout()
    #plt.figure(i).suptitle('%s: extrema are %s \n Left:  %s \n Right:  %s' %(name, max_locations, left_lane.recent_xfitted[-1], right_lane.recent_xfitted[-1]), fontsize=16)
    plt.figure(i).suptitle('%s' %(name), fontsize=16)
    plt.figure(i).add_subplot(2,2,1).set_title('Output', fontsize=20)
    plt.imshow(result)
    plt.figure(i).add_subplot(2,2,2).set_title('Warped Mask', fontsize=20)
    plt.imshow(warped)
    plt.plot(histogram)
    plt.figure(i).add_subplot(2,2,3).set_title('Found Points', fontsize=20)
    plt.imshow(found_points)
    plt.figure(i).add_subplot(2,2,4).set_title('Combined', fontsize=20)
    plt.imshow(combined)
    
    # Plots the left and right polynomials on the lane lines
    #plt.plot(left_fitx, ploty,  color='orange')
    #plt.plot(right_fitx, ploty, color='yellow')

    i+=1
    
#print('Left lane coeffs:  %s' %left_lane.recent_coeffs)
#print('Right lane coeffs:  %s' %right_lane.recent_coeffs)
#print('Left lane xs:  %s' %left_lane.recent_xfitted)
#print('Right lane xs:  %s' %right_lane.recent_xfitted)

reading straight_lines1.jpg


# Run on video files

In [14]:
left_lane = Line()
right_lane = Line()
white_output = './output_images/output_project_video.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
clip1 = VideoFileClip("./project_video.mp4")
#clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(12,14)
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video ./output_images/output_project_video.mp4
[MoviePy] Writing video ./output_images/output_project_video.mp4



 24%|██▍       | 303/1261 [03:49<12:10,  1.31it/s]


 48%|████▊     | 605/1261 [07:43<08:55,  1.22it/s]


 72%|███████▏  | 907/1261 [11:36<04:35,  1.28it/s]


 96%|█████████▌| 1205/1261 [15:29<00:41,  1.34it/s]


100%|█████████▉| 1260/1261 [16:11<00:00,  1.31it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./output_images/output_project_video.mp4 

CPU times: user 13min 51s, sys: 2.83 s, total: 13min 53s
Wall time: 16min 15s


# challlenge video

In [ ]:
left_lane = Line()
right_lane = Line()
white_output = './output_images/output_challenge_video.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
clip1 = VideoFileClip("./challenge_video.mp4")
#clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(12,14)
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

# harder challenge

In [15]:
left_lane = Line()
right_lane = Line()
white_output = './output_images/output_harder_challenge_video.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
clip1 = VideoFileClip("./harder_challenge_video.mp4")
#clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(12,14)
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video ./output_images/output_harder_challenge_video.mp4
[MoviePy] Writing video ./output_images/output_harder_challenge_video.mp4



 25%|██▌       | 303/1200 [04:05<12:05,  1.24it/s]


 50%|█████     | 605/1200 [08:16<07:32,  1.31it/s]


 76%|███████▌  | 907/1200 [12:32<03:56,  1.24it/s]


100%|█████████▉| 1199/1200 [16:30<00:00,  1.22it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./output_images/output_harder_challenge_video.mp4 

CPU times: user 13min 35s, sys: 2.6 s, total: 13min 38s
Wall time: 16min 35s


# Create test images from the challenge videos

In [17]:
#overpass = VideoFileClip("./challenge_video.mp4").subclip(4.2,4.3)
#overpass.write_videofile('test_videos/overpass.mp4', audio=False)
#pavement = VideoFileClip("./challenge_video.mp4").subclip(6,6.1)
#pavement.write_videofile('test_videos/pavement.mp4', audio=False)
#leaves = VideoFileClip("./harder_challenge_video.mp4").subclip(3,3.1)
#leaves.write_videofile('test_videos/leaves.mp4', audio=False)
#shadows = VideoFileClip("./harder_challenge_video.mp4").subclip(7,7.1)
#shadows.write_videofile('test_videos/shadows.mp4', audio=False)
colorchange = VideoFileClip("./challenge_video.mp4").subclip(0,.8)
colorchange.write_videofile('test_videos/challenge_start_video.mp4', audio=False)
#names =['overpass', 'pavement', 'leaves', 'shadows']
names = ['challenge_start_video']
for fname in names:
    vidcap = cv2.VideoCapture('test_videos/%s.mp4' %(fname))
    print('reading image')
    success,image = vidcap.read()
    count = 0
    success = True
    while success:
      cv2.imwrite('test_images/challenge/%s_frame%d.jpg' %(fname, count), image)     
      success,image = vidcap.read()
      print('Read a new frame: ', success)
      count += 1


OSError: [WinError 6] The handle is invalid

f, axes = plt.subplots(4, 2, figsize=(24, 12))
f.tight_layout()

axes[0,0].imshow(undistorted)
axes[0,0].set_title('Original Image', fontsize=20)

axes[0,1].imshow(gray)
axes[0,1].set_title('Grayscale', fontsize=20)

axes[1,0].imshow(blur_gray)
axes[1,0].set_title('Blurred Grayscale', fontsize=20)

axes[1,1].imshow(x_binary)
axes[1,1].set_title('SobelX Binary', fontsize=20)

axes[2,0].imshow(y_binary)
axes[2,0].set_title('SobelY Binary', fontsize=20)

axes[2,1].imshow(mag_binary)
axes[2,1].set_title('Magnitude Binary', fontsize=20)

axes[3,0].imshow(dir_interesting)
axes[3,0].set_title('Direction Binary', fontsize=20)

axes[3,1].imshow(Canny_binary)
axes[3,1].set_title('Canny Binary', fontsize=20)


fig, color_axes = plt.subplots(3, 2, figsize=(24, 9))
fig.tight_layout()

color_axes[0,0].imshow(S)
color_axes[0,0].set_title('S', fontsize=20)

color_axes[0,1].imshow(s_binary)
color_axes[0,1].set_title('S binary', fontsize=20)

color_axes[1,0].imshow(H)
color_axes[1,0].set_title('H', fontsize=20)

color_axes[1,1].imshow(h_binary)
color_axes[1,1].set_title('H binary', fontsize=20)

color_axes[2,0].imshow(L)
color_axes[2,0].set_title('L', fontsize=20)

color_axes[2,1].imshow(l_binary)
color_axes[2,1].set_title('L binary', fontsize=20)

fig2, axes = plt.subplots(2, 2, figsize=(24, 9))
fig2.tight_layout()

axes[0,0].imshow(white)
axes[0,0].set_title('White Lines', fontsize=20)

axes[0,1].imshow(yellow)
axes[0,1].set_title('Yellow Lines', fontsize=20)

axes[1,0].imshow(image)
axes[1,0].set_title('Original', fontsize=20)

axes[1,1].imshow(combined)
axes[1,1].set_title('Combined', fontsize=20)


    #unwarped_left_lane = (nearest_left - (dst[0][3][0]+dst[0][0][0])//2) * ((src[0][3][0]-src[0][0][0])/(dst[0][3][0]-dst[0][0][0])) + undistorted.shape[1]//2
    #unwarped_right_lane = (nearest_right - (dst[0][3][0]+dst[0][0][0])//2) * (src[0][3][0]-src[0][0][0])/(dst[0][3][0]-dst[0][0][0]) + undistorted.shape[1]//2
    #mperpix = 3.7/(unwarped_right_lane - unwarped_left_lane)
    #offset = (undistorted.shape[1]//2 - (unwarped_right_lane + unwarped_left_lane)//2) * mperpix 
    #offset = round(offset,2)
    
    

